In [ ]:
!pip install crewai duckduckgo-search langchain_google_vertexai html2text

In [11]:
from langchain_google_vertexai import ChatVertexAI
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import Tool
from langchain_core.tools import tool
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from langchain_google_vertexai.vision_models import VertexAIImageGeneratorChat
from langchain_core.messages import AIMessage, HumanMessage
from crewai import Agent, Task, Crew, Process
import datetime
import base64
import io
from PIL import Image
from IPython.display import Markdown

In [2]:
llm = ChatVertexAI(
    model_name="gemini-1.5-flash"
)

In [34]:
ddg_wrapper = DuckDuckGoSearchAPIWrapper(region="uk-en", time="w", max_results=100)
search = DuckDuckGoSearchResults(api_wrapper=ddg_wrapper, backend="text")

search_tool = Tool(
    name="search",
    description="Search the internet for relevant information",
    func=search.run,
)

In [35]:
@tool("date_tool", return_direct=True)
def date_tool():
    """Today's date"""
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    return f"Today's date is {today}."

@tool("arxiv_tool", return_direct=False)
def arxiv_tool():
    """Look up latest academic journal articles."""
    loader = AsyncHtmlLoader("https://arxiv.org/list/cs/new?skip=0&show=50")
    docs = loader.load()
    html2text = Html2TextTransformer(ignore_links=False)
    docs_transformed = html2text.transform_documents(docs)
    return docs_transformed

@tool("url_tool", return_direct=False)
def url_tool(urls: list[str]):
    """Download webpage content directly from URLs."""
    loader = AsyncHtmlLoader(urls)
    docs = loader.load()
    html2text = Html2TextTransformer(ignore_links=False)
    docs_transformed = html2text.transform_documents(docs)
    return docs_transformed

In [37]:
data_science_researcher = Agent(
    role="Data Science Researcher",
    goal="Monitor and analyse the latest development in Data Science by researching papers posted on the HuggingFace or ArXiv. "
         "Presents complex concepts in layman terms so that everyone understands. ",
    backstory="Being an experienced data science researcher, this agent has extensive experience in "
              "conducting internet serach to identify latest trends in Data Science, Artificial Intelligence, Machine Learning. "
              "This agent always reads journal papers, articles, websites to stay up-to-date. "
              "This agent MUST use the arxiv_tool to read latest academic papers. "
              "This agent will always pass on the findings alongside the source URLs. "
              "The research must be published within the previous seven days. Today's date can be found using tool. "
              "Use tools if needed and give final answer once the  right information has been collected. ",
    llm=llm,
    verbose=True,
    tools = [
        date_tool,
        search_tool,
        arxiv_tool,
        url_tool
    ],
    allow_delegation=False,
)

social_media_writer = Agent(
    role="Social Media Writer",
    goal="Create trending social media post on LinkedIn on Data Science, AI and Machine Learning topics "
         "based on findings from the Data Science Researcher.",
    backstory="This agent specialises in writing trending LinkedIn posts, "
              "understands how best to present information to both technical and non-technical users. "
              "This agent can instruct coworkers to provide relevant information to accomplish tasks. "
              "You will use tools if needed and give final answer once you have collected the right information. ",
    llm=llm,
    verbose=True,
    tools = [
        url_tool
    ],
    allow_delegation=True,
)

In [38]:
data_science_research_task = Task(
    description=(
        "Continuously monitor latest development on these topic: {topic}. "
        "You will read the academic journal articles and return ONE article which is most relevant. "
        "The article you return must be an academic journal paper relating to one of the chosen topics. "
        "The URL must be extracted using output of the tool. "
    ),
    expected_output=(
        "Summary of findings and the corresponding source URL, which links to the original article. "
    ),
    agent=data_science_researcher,
)

draft_post_task = Task(
    description=(
        "Draft a trending LinkedIn post on one of these topics: {topic}. "
        "Write in simple English, so gthat it can be understood by both technical and non-technical audiences."
        "Summarise what the developments are and suggest how the they can be used in real-world scenario, solving actual business problems. "
        "The source URLs provided by the Data Science Researcher must be included in the draft post. "
        "Placeholders such as [NAME] must not be used. "
        "You can add emojis to make the post more interesting. "
        "You should also use suitable hashtags to make the post more engaging. "
    ),
    expected_output=(
        "An interesting and engaging social media post on LinkedIn. "
        "You must include exactly one source URL which links to the journal article. "
    ),
    agent=social_media_writer,
    context=[
        data_science_research_task,
    ],
)

verification_task = Task(
    description=(
        "Download contents from the source URL using tool to ensure it is the correct link to use, given the draft LinkedIn post. "
        "The link should lead to the full article or the journal paper. It should not simply be a landing page without details. "
        "If they do not seem to align, or the URL doesn't work, you must request this to be corrected. "
    ),
    expected_output=(
        "Verify that the content and source URL are correct and aligned with one another. URL must be in the draft post. "
    ),
    agent=social_media_writer,
    context=[
        draft_post_task,
    ],
)

image_prompt_task = Task(
    description=(
        "Based on the draft LinkedIn post, write a prompt to generate an image. "
    ),
    expected_output=(
        "An image prompt which is suits the contents of the draft post."
    ),
    agent=social_media_writer,
    context=[
        verification_task,
    ],
)

In [39]:
# Define the crew with agents and tasks
crew = Crew(
    agents=[
        data_science_researcher,
        social_media_writer,
    ],
    tasks=[
        data_science_research_task,
        draft_post_task,
        verification_task,
        image_prompt_task,
    ],
    process=Process.hierarchical,
    manager_llm=llm,
    verbose=1,
    memory=True,
    full_output=True,
    embedder={
        "provider": "vertexai",
        "config":{
            "model": 'textembedding-gecko'
        }
    }
)

2024-06-21 10:00:03,719 - 140608496928576 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [40]:
result = crew.kickoff(
    inputs={
        "topic": "Data Science, Machine Learning, Generative AI, LLM, NLP, Graph Theory, Multiagent System"
    }
)

 [DEBUG]: Working Agent: Crew Manager


> Entering new CrewAgentExecutor chain...
Thought: I need to delegate the task of finding the most relevant academic journal article to the Data Science Researcher. 

Action: Delegate work to co-worker
Action Input: {
  "coworker": "Data Science Researcher",
  "task": "Find the most relevant academic journal article related to the topics: Data Science, Machine Learning, Generative AI, LLM, NLP, Graph Theory, Multiagent System",
  "context": "Please read recent academic journal articles related to these topics and provide the most relevant article. The article should be an academic journal paper and relate to one of the chosen topics. Please also extract the URL of the article."
}


> Entering new CrewAgentExecutor chain...
Thought: I need to find the latest academic journal articles related to the specified topics. I will use the arxiv_tool to search for relevant papers.

Action: arxiv_tool
Action Input: {"query": "Data Science, Machine Learning,

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  5.56it/s]


 

[Document(page_content='Skip to main content\n\n[](https://www.cornell.edu/)\n\nWe gratefully acknowledge support from the Simons Foundation, [member\ninstitutions](https://info.arxiv.org/about/ourmembers.html), and all\ncontributors. [Donate](https://info.arxiv.org/about/donate.html)\n\n[]({url_path\\(\'ignore_me\'\\)})\n\n[](/) > [cs](/list/cs/recent)\n\n[Help](https://info.arxiv.org/help) | [Advanced Search](https://arxiv.org/search/advanced)\n\nAll fields Title Author Abstract Comments Journal reference ACM classification\nMSC classification Report number arXiv identifier DOI ORCID arXiv author ID\nHelp pages Full text\n\nSearch\n\n[](https://arxiv.org/)\n\n[ ](https://www.cornell.edu/)\n\nopen search\n\nGO\n\nopen navigation menu\n\n## quick links\n\n  * [Login](https://arxiv.org/login)\n  * [Help Pages](https://info.arxiv.org/help)\n  * [About](https://info.arxiv.org/about)\n\n# Computer Science\n\n  * New submissions\n  * [Cross-lists](/list/cs/new?skip=750&show=50#item795)\n

Fetching pages: 100%|##########| 1/1 [00:00<00:00, 33.13it/s]


 

[Document(page_content='Skip to main content\n\n[](https://www.cornell.edu/)\n\nWe gratefully acknowledge support from the Simons Foundation, [member\ninstitutions](https://info.arxiv.org/about/ourmembers.html), and all\ncontributors. [Donate](https://info.arxiv.org/about/donate.html)\n\n[]({url_path\\(\'ignore_me\'\\)})\n\n[](/) > [cs](/list/cs/recent)\n\n[Help](https://info.arxiv.org/help) | [Advanced Search](https://arxiv.org/search/advanced)\n\nAll fields Title Author Abstract Comments Journal reference ACM classification\nMSC classification Report number arXiv identifier DOI ORCID arXiv author ID\nHelp pages Full text\n\nSearch\n\n[](https://arxiv.org/)\n\n[ ](https://www.cornell.edu/)\n\nopen search\n\nGO\n\nopen navigation menu\n\n## quick links\n\n  * [Login](https://arxiv.org/login)\n  * [Help Pages](https://info.arxiv.org/help)\n  * [About](https://info.arxiv.org/about)\n\n# Computer Science\n\n  * New submissions\n  * [Cross-lists](/list/cs/new?skip=750&show=50#item795)\n

...

Thought: I need to understand the key findings of the paper 'ABNet: Attention BarrierNet for Safe and Scalable Robot Learning' to draft a trending LinkedIn post about it. I'll use the arxiv_tool to get the abstract and then use the url_tool to get the full paper to extract the relevant information.

Action: arxiv_tool
Action Input: {"query": "ABNet: Attention BarrierNet for Safe and Scalable Robot Learning"} 

[Document(page_content='Skip to main content\n\n[](https://www.cornell.edu/)\n\nWe gratefully acknowledge support from the Simons Foundation, [member\ninstitutions](https://info.arxiv.org/about/ourmembers.html), and all\ncontributors. [Donate](https://info.arxiv.org/about/donate.html)\n\n[]({url_path\\(\'ignore_me\'\\)})\n\n[](/) > [cs](/list/cs/recent)\n\n[Help](https://info.arxiv.org/help) | [Advanced Search](https://arxiv.org/search/advanced)\n\nAll fields Title Author Abstract Comments Journal reference ACM classification\nMSC classification Report number arXiv identifier DOI

2024-06-21 10:03:23,391 - 140608496928576 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--50c0525db2bd8cb17c7118dd634d92c54552f7c286ab45fe44fb94b5dce490c4


Thought: I need to verify if the provided URL leads to the full paper 'ABNet: Attention BarrierNet for Safe and Scalable Robot Learning'. I will use the url_tool to download the webpage content and check if it matches the expected paper title.

Action: url_tool
Action Input: {"urls": ["https://arxiv.org/abs/2406.13025"]} 

[Document(page_content='Skip to main content\n\n[](https://www.cornell.edu/)\n\nWe gratefully acknowledge support from the Simons Foundation, [member\ninstitutions](https://info.arxiv.org/about/ourmembers.html), and all\ncontributors. [Donate](https://info.arxiv.org/about/donate.html)\n\n[]({url_path\\(\'ignore_me\'\\)})\n\n[](/) > [cs](/list/cs/recent) > arXiv:2406.13025\n\n[Help](https://info.arxiv.org/help) | [Advanced Search](https://arxiv.org/search/advanced)\n\nAll fields Title Author Abstract Comments Journal reference ACM classification\nMSC classification Report number arXiv identifier DOI ORCID arXiv author ID\nHelp pages Full text\n\nSearch\n\n[](https://a

2024-06-21 10:03:41,123 - 140608496928576 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--f266af2eff44a143a495e95c601f8c89c2ad9681786ae4491f7e2fac0c63e065


Thought: I need to understand the key features of ABNet, especially its attention mechanisms, safety, and scalability, to create a visual representation for LinkedIn. I'll use the arxiv_tool to get the abstract and then use the url_tool to get the full paper for a deeper understanding. 

Action: arxiv_tool
Action Input: {"query": "ABNet: Attention BarrierNet for Safe and Scalable Robot Learning"} 

[Document(page_content='Skip to main content\n\n[](https://www.cornell.edu/)\n\nWe gratefully acknowledge support from the Simons Foundation, [member\ninstitutions](https://info.arxiv.org/about/ourmembers.html), and all\ncontributors. [Donate](https://info.arxiv.org/about/donate.html)\n\n[]({url_path\\(\'ignore_me\'\\)})\n\n[](/) > [cs](/list/cs/recent)\n\n[Help](https://info.arxiv.org/help) | [Advanced Search](https://arxiv.org/search/advanced)\n\nAll fields Title Author Abstract Comments Journal reference ACM classification\nMSC classification Report number arXiv identifier DOI ORCID arXiv

In [41]:
Markdown(draft_post_task.output.exported_output)

**Exciting news in Machine Learning for Robotics! 🚀**

Safe learning is crucial for robots operating in real-world environments where mistakes can have serious consequences. 🤖 

Traditional **barrier-based** methods, while promising, have limitations like scalability and difficulty in training. 😓 

A new paper, **ABNet: Attention BarrierNet for Safe and Scalable Robot Learning**, proposes an innovative solution. 🤔

**ABNet** leverages **attention mechanisms** to build larger, safer models incrementally. This approach allows the model to learn from different features and focus on specific aspects of the observation, making training much easier. 💪

ABNet's strengths are evident in its applications, such as:
- 2D robot obstacle avoidance 
- Safe robot manipulation 
- Vision-based autonomous driving 

ABNet demonstrates **improved robustness and safety guarantees** compared to existing models. 🛡️

These developments have the potential to revolutionize the use of robots in various industries, solving real-world problems like:
- **Warehouse automation:** Safer robots for efficient and reliable product handling.
- **Healthcare:** Robots assisting with complex surgeries and patient care with increased safety.
- **Autonomous vehicles:** Safer self-driving cars for improved road safety.

**This research is a significant step forward in making robots more intelligent, adaptable, and safe.** 🤝

**Read the full paper here:** [https://arxiv.org/abs/2406.13025](https://arxiv.org/abs/2406.13025) 

#MachineLearning #Robotics #AI #SafeAI #RobotLearning #ABNet #AutonomousSystems #Innovation #FutureTech

# Vertex AI Imagen

Draw a picture to go with the content. Reference URL: https://cloud.google.com/vertex-ai/generative-ai/docs/image/overview

In [21]:
def generate_image(prompt:str, filepath:str):
    """Generate an image based on the given prompt and save it locally"""
    
    generator = VertexAIImageGeneratorChat()
    messages = [HumanMessage(content=[prompt])]
    response = generator.invoke(messages)
    generated_image = response.content[0]


    # Parse response object to get base64 string for image
    img_base64 = generated_image["image_url"]["url"].split(",")[-1]

    # Convert base64 string to Image
    img = Image.open(io.BytesIO(base64.decodebytes(bytes(img_base64, "utf-8"))))

    # view Image
    img.save(filepath, "JPEG")

In [22]:
generate_image(image_prompt_task.output.exported_output, "image.jpg")